In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df_train = pd.read_csv('..//data/train.csv')

# Handling Missing Values

In [55]:
for col in df_train.columns:
    miss_percentage = df_train[col].isnull().sum() / df_train.shape[0] * 100
    print(f"{col}: {miss_percentage:.2f}%")

Id: 0.00%
MSSubClass: 0.00%
MSZoning: 0.00%
LotFrontage: 17.74%
LotArea: 0.00%
Street: 0.00%
Alley: 93.77%
LotShape: 0.00%
LandContour: 0.00%
Utilities: 0.00%
LotConfig: 0.00%
LandSlope: 0.00%
Neighborhood: 0.00%
Condition1: 0.00%
Condition2: 0.00%
BldgType: 0.00%
HouseStyle: 0.00%
OverallQual: 0.00%
OverallCond: 0.00%
YearBuilt: 0.00%
YearRemodAdd: 0.00%
RoofStyle: 0.00%
RoofMatl: 0.00%
Exterior1st: 0.00%
Exterior2nd: 0.00%
MasVnrType: 59.73%
MasVnrArea: 0.55%
ExterQual: 0.00%
ExterCond: 0.00%
Foundation: 0.00%
BsmtQual: 2.53%
BsmtCond: 2.53%
BsmtExposure: 2.60%
BsmtFinType1: 2.53%
BsmtFinSF1: 0.00%
BsmtFinType2: 2.60%
BsmtFinSF2: 0.00%
BsmtUnfSF: 0.00%
TotalBsmtSF: 0.00%
Heating: 0.00%
HeatingQC: 0.00%
CentralAir: 0.00%
Electrical: 0.07%
1stFlrSF: 0.00%
2ndFlrSF: 0.00%
LowQualFinSF: 0.00%
GrLivArea: 0.00%
BsmtFullBath: 0.00%
BsmtHalfBath: 0.00%
FullBath: 0.00%
HalfBath: 0.00%
BedroomAbvGr: 0.00%
KitchenAbvGr: 0.00%
KitchenQual: 0.00%
TotRmsAbvGrd: 0.00%
Functional: 0.00%
Fireplaces: 

In [53]:
# Check if the missingness are significanltly correlated with SalePrice
df_copy = df_train.copy()

for col in df_copy.columns:
    df_copy[col] = df_copy[col].isnull().astype(int)  # 1 for missing, 0 for not missing

df_missing_corr = df_copy.loc[:, df_copy.nunique() > 1]  # Keep only columns with more than 1 unique value
df_missing_corr['SalePrice'] = df_train['SalePrice']    
correlations = df_missing_corr.corr()['SalePrice'].sort_values(ascending=False)
meaningful_features = correlations[correlations.abs() > 0.3]
print(correlations)

SalePrice       1.000000
Fence           0.172184
Alley           0.123611
MiscFeature     0.072298
MasVnrArea      0.051933
LotFrontage     0.004087
Electrical     -0.004424
PoolQC         -0.093708
BsmtFinType2   -0.145274
BsmtExposure   -0.150231
BsmtFinType1   -0.152829
BsmtCond       -0.152829
BsmtQual       -0.152829
GarageType     -0.236832
GarageYrBlt    -0.236832
GarageFinish   -0.236832
GarageQual     -0.236832
GarageCond     -0.236832
MasVnrType     -0.367456
FireplaceQu    -0.471908
Name: SalePrice, dtype: float64


/var/folders/h0/2sdvn_yn51z72lw90rbmwpgm0000gn/T/ipykernel_2969/264689648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing_corr['SalePrice'] = df_train['SalePrice']


In [ ]:
# Drop those columns whose correlation is weak and have missing > 10%
columns_to_drop = []
for col in df_train.columns:
    missing_percentage = df_train[col].isnull().sum() / df_train.shape[0] * 100
    if col not in meaningful_features and missing_percentage > 10:
        columns_to_drop.append(col)
df_train.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
# Treatment of meaningful features with high missing values
high_missing_features = ['MasVnrType', 'FireplaceQu']  # List of important features
df_train[high_missing_features] = df_train[high_missing_features].fillna('None')

In [69]:
# Identify those column with missing values > 0
for col in df_train.columns.sort_values():
    miss_percentage = df_train[col].isnull().sum() / df_train.shape[0] * 100
    if miss_percentage > 0:
        print(f"{col}: {miss_percentage:.2f}%")

BsmtCond: 2.53%
BsmtExposure: 2.60%
BsmtFinType1: 2.53%
BsmtFinType2: 2.60%
BsmtQual: 2.53%
Electrical: 0.07%
GarageCond: 5.55%
GarageFinish: 5.55%
GarageQual: 5.55%
GarageType: 5.55%
GarageYrBlt: 5.55%
MasVnrArea: 0.55%


In [ ]:
# Checked data description and found those features,
# where None means something and can be used instead of null values.
low_missing_features = ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
                        'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt',
                        'MasVnrArea']
df_train[low_missing_features] = df_train[low_missing_features].fillna('None')

In [75]:
# Treatment of Electrical --> Note the percentage of missing value is very low
# print(f'Number of missing values = {df_train['Electrical'].isnull().sum()}')
# df_train['Electrical'].value_counts()
#!!!!! There is only one missing value in the Electrical column, so we can fill it with the mode !!!!!
df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])

# Handling multicollinearity

In [85]:
corr = df_train.select_dtypes(include='number').corr()
corr[corr.iloc[:, :] > 0.5]

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MSSubClass,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotArea,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverallQual,NaN,NaN,NaN,1.000000,NaN,0.572323,0.550684,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.790982
OverallCond,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YearBuilt,NaN,NaN,NaN,0.572323,NaN,1.000000,0.592855,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.522897
YearRemodAdd,NaN,NaN,NaN,0.550684,NaN,0.592855,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.507101
BsmtFinSF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtFinSF2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtUnfSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
